# Average Mode

The script demonstrates the use of average mode. The first sections connects & configures the system 
and sets the waveform generator. The latter sections define the average mode setting & digitizer sampling operation. 
The generator is configured on Channel-1, the signal is defined through a task table which consists of single cycle of "sin" wave followed by a "dc" quiet time.
An additional marker is defined on Marker-1 of channel 1 - currently in the half time of the active signal.

The digitizer is also configured on the input channel-1, so some loopback is required for the testing.
Starting from section: 'Average Mode Parameters', the average mode setting are defined 
and consists of the following:

#### Average Count up 1-16M
AvgCount=8000000
#### Frame Size 1-10200
frame_size = 9600 
####  number_of_frames =1 : Should be fixed one.

In order to repeat tests with different parameters, you only need to replay sections starting from section: 'Average Mode Parameters'.
A background parameter called the AverageMSBShift - is set to be the log2 of the AvgCount parameter since the summing accumlators are limted to 36 bit. 

The resulting data is displayed in a "raw" & normalized forms at the end.




In [1]:
import os
import sys
import gc
srcpath = os.path.realpath('../../SourceFiles') # TeraBAT unit
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from tevisainst import TEVisaInst

In [ ]:
import numpy as np
import time as timelib
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt

lib_dir_path_= None #r'C:\Work_Svn\ProteusAwg_alice\x64\Debug'


In [ ]:
# Connect to instrument

inst = None
admin = None

conn_msg_str = ''

def disconnect():
    global inst
    global admin
    if inst is not None:
        try:
            inst.close_instrument()            
        except:
            pass
        inst = None
    if admin is not None:
        try:
            admin.close_inst_admin()
        except:
            pass
        admin = None
    gc.collect()

def connect_to_pxi_slot(slot_id):
    global inst
    global admin
    try:
        disconnect()        
        print("Trying to connect to PXI-slot:" + str(slot_id))
        admin = TepAdmin(lib_dir_path= lib_dir_path_)
        admin.open_inst_admin()        
        inst = admin.open_instrument(slot_id, reset_hot_flag=True)
    except:
        pass

def connect_to_lan_server(ip_address):
    global inst
    try:
        disconnect()
        print("Trying to connect to IP:" + ip_address)
        inst = TEVisaInst(ip_address, port=5025)
    except:
        pass

def connect(ipaddr_or_slotid):
    try:
        disconnect()
        if isinstance(ipaddr_or_slotid, str) and '.' in ipaddr_or_slotid:
            connect_to_lan_server(ipaddr_or_slotid)
        else:
            connect_to_pxi_slot(ipaddr_or_slotid)
    except:
        pass

addr_textbox = widgets.Text(
    value='',
    placeholder='IP Address or Slot Id',
    description='Address:',
    disabled=False
)

output = widgets.Output()

def print_conn_message(msg_str):
    global output
    global conn_msg_str
    with output:
        if len(conn_msg_str) > 0:
            print('\r', end='')
            print(' ' * len(conn_msg_str), end='')
            print('\r', end='')
        conn_msg_str = str(msg_str)
        if len(conn_msg_str) > 0:
            print(conn_msg_str, end='')

def on_button_connect_clicked(conn_str):
    global inst
    global addr_textbox
    print_conn_message('')
    disconnect()
    if addr_textbox is not None:
        conn_str = addr_textbox.value 
        if conn_str:
            print_conn_message('Connecting to {0} ..'.format(conn_str))
            connect(conn_str)
            print_conn_message('')
    if inst is not None:
        idn_str = inst.send_scpi_query('*IDN?')
        model = inst.send_scpi_query(":SYST:iNF:MODel?")
        if idn_str:
            prnmsg = 'Connected to: ' + idn_str.strip()
            print_conn_message(prnmsg)
            #print_conn_message('Connected to: ' + idn_str.strip())
            
    else:
        print_conn_message('Instrument not connected..')
    


In [ ]:
connect("192.168.0.117")   #TeraBAT unit



if inst is not None:
        idn_str = inst.send_scpi_query('*IDN?')
        model = inst.send_scpi_query(":SYST:iNF:MODel?")
        if idn_str:
            prnmsg = 'Connected to: ' + idn_str.strip()
            print(prnmsg)
else:
    print("Not connected")

pid = os.getpid()
print('process id {0}'.format(pid))


paranoia_level =2

model = inst.send_scpi_query(":SYST:iNF:MODel?")
print("\nModel: " + model)
# Infer the natural DAC waveform format
if 'P9082' in model:
    dac_mode = 8
else:
    dac_mode = 16
print("DAC waveform format: {0} bits-per-point".format(dac_mode))

if dac_mode == 16:
    max_dac = 65535
    data_type = np.uint16 
else:
    max_dac = 255
    data_type = np.uint8 
    
half_dac = max_dac / 2.0

sampleRateDAC = 8640e6

inst.send_scpi_cmd(":INST:CHAN 1")
inst.send_scpi_cmd(':OUTP OFF')
inst.send_scpi_cmd(':FUNC:MODE ARB')
inst.send_scpi_cmd(':TRAC:DEL:ALL')
inst.send_scpi_cmd(':TASK:ZERO:ALL')
inst.send_scpi_cmd(":INST:CHAN 3")
inst.send_scpi_cmd(':OUTP OFF')
inst.send_scpi_cmd(':FUNC:MODE ARB')
inst.send_scpi_cmd(':TRAC:DEL:ALL')
inst.send_scpi_cmd(':TASK:ZERO:ALL')
inst.send_scpi_cmd(':DIG:CHAN:SEL 1')

inst.send_scpi_cmd(':DIG:ACQuire:ZERO 1,-1')

inst.send_scpi_cmd("*CLS; *RST")

print(dac_mode)
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

In [ ]:

inst.send_scpi_cmd(":FREQ:RAST {0}".format(sampleRateDAC))
inst.send_scpi_cmd(":INIT:CONT ON")

sclk = inst.send_scpi_query(":FREQ:RAST?")
print("Sample Clk: " + sclk + "MHz")
    


## Define Seg 1 - (IDLE DC) Level

In [ ]:
cycleLen = 2048
numCycles = 1
segLenDCL1 = cycleLen * numCycles

# Build waveform (triangle wave)
y = np.ones(segLenDCL1)* half_dac
y = y.astype(data_type)
y.reshape(-1)

print(y.itemsize)

# Set Data as Task 1
#inst.send_scpi_cmd(':TRAC:DEF 1, {0}'.format(segLen*y.itemsize))
inst.send_scpi_cmd(":INST:CHAN 1")
inst.send_scpi_cmd(':TRAC:DEF 1, {0}'.format(segLenDCL1)) #*y.itemsize))
inst.send_scpi_cmd(':TRAC:SEL 1')
# download the waveform to the selected Task
inst.write_binary_data(':TRAC:DATA', y)
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

plt.plot(y)


In [ ]:
# Set Data as Task 1
#inst.send_scpi_cmd(':TRAC:DEF 1, {0}'.format(segLen*y.itemsize))
#inst.send_scpi_cmd(':TRAC:DEF 1, {0}'.format(segLen)) #*y.itemsize))
#inst.send_scpi_cmd(':TRAC:SEL 1')
# download the waveform to the selected Task
#inst.write_binary_data(':TRAC:DATA', y)
#resp = inst.send_scpi_query(':SYST:ERR?')
#print(resp)

## Define Seg 2 - Chirp

In [ ]:
print('Build wave-data and download it to Proteus device.')
# Build a chirp segment that will be used as innput for the digitizer
paranoia_level = 2
# Build wave-data and markers-data for 12 segments

print('Build wave-data and markers-data ..')
seg_wave_points =  896 # 100 ns of the total 5184 # 600 ns period

#-----------------CH1 AWG Waveform-----------------------------------
amp = 1
max_dac=65535
half_dac=max_dac/2
data_type = np.uint16
#Set wveform 
segLen = 173 # 173 ~20ns pulse, Full period seg_wave_points=5184 equals to 600 ns at 8.64 GS/s
segLenDC = seg_wave_points-segLen #DC
#sampleRateDAC=freq
#Fn1=3.6e9
Fn1=0.2e9
Fcrp=1e6
T = ((segLen)/(sampleRateDAC))
time = np.arange(0, segLen) / sampleRateDAC
CRPWave = amp*chirp(time, f0=Fn1-Fcrp, f1=Fn1+Fcrp, t1=T, method='linear') 
#w = np.hanning(len(CRPWave))
w = 1 #uncomment for no window
dacWave = ((w * CRPWave) + 1.0) * half_dac   
dacWave = np.round(dacWave)
dacWave = np.clip(dacWave, 0, max_dac)
dacWave = dacWave.astype(data_type)
#Set DC
dacWaveDC = np.zeros(segLenDC)       
dacWaveDC = dacWaveDC+(max_dac/2)
dacWaveDC = dacWaveDC.astype(np.uint16)
    
dacWave = np.concatenate([dacWave, dacWaveDC])
print("DACWAVE LEN")
print(len(CRPWave))
#print(len(w))
print(len(dacWave))
#-----------------end CH1 AWG Waveform--------------------------------------
  
plt.plot(dacWave)

In [ ]:
# Download it to segment 1 of channel 1

inst.send_scpi_cmd(":INST:CHAN 1")
#inst.send_scpi_cmd( ":TRAC:DEF 2," + str(len(dacWave)), paranoia_level)
inst.send_scpi_cmd( ":TRAC:DEF 2, " + str(seg_wave_points), paranoia_level)
inst.send_scpi_cmd( ":TRAC:SEL 2", paranoia_level)

inst.write_binary_data(':TRAC:DATA', dacWave)
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

## 2nd DC - Remainder of CH1 (Seg3)

In [ ]:
#cycleLen = 5184-864 # total period size minus the first 100 ns
numCycles = 1
segLenR = 4224 # 4096 #Divisible 64. #5184 #5184-864 = 4320

# Build waveform (triangle wave)
y = np.ones(segLenR)* half_dac
y = y.astype(data_type)
y.reshape(-1)

print(y.itemsize)

# Set Data as Task 1
inst.send_scpi_cmd(":INST:CHAN 1")
#inst.send_scpi_cmd(':TRAC:DEF 1, {0}'.format(segLen*y.itemsize))
inst.send_scpi_cmd(':TRAC:DEF 3, {0}'.format(segLenR)) #*y.itemsize))
inst.send_scpi_cmd(':TRAC:SEL 3')
# download the waveform to the selected Task
inst.write_binary_data(':TRAC:DATA', y)
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

plt.plot(y)


## Define Seg 4 - CW

In [ ]:

print('Build wave-data for channel 3 and download it to Proteus device.')

# Build a CW segment that will be used as innput for the digitizer
paranoia_level = 2
# Build wave-data and markers-data for 12 segments

print('Build wave-data and markers-data ..')
seg_wave_points_CH3 = seg_wave_points + segLenR #5184 # 600 ns period

#-----------------CH3 AWG Waveform-----------------------------------
amp = 1
max_dac=65535
half_dac=max_dac/2
data_type = np.uint16
#Set wveform 
segLen3 = 4800 # 173 ~20ns pulse, Full period seg_wave_points=5184 equals to 600 ns at 8.64 GS/s

#sampleRateDAC=freq

Fn3=4.2e9
cycles = Fn3 * segLen3/sampleRateDAC
time = np.linspace(0, segLen3-1, segLen3)
#time = np.arange(0, segLen3) / sampleRateDAC
omega = 2 * np.pi * cycles
print("segLen3 = " + str(segLen3))
print("cycles = " + str(cycles))
CWWave3 = amp*(np.sin(omega*time/segLen3))
#w = np.hanning(len(CWWave3))
w = 1 #uncomment for no window
dacWave3 = ((w * CWWave3) + 1.0) * half_dac    
dacWave3 = np.round(dacWave3)
dacWave3 = np.clip(dacWave3, 0, max_dac)
dacWave3 = dacWave3.astype(data_type)


#print("DACWAVE3 LEN")
print(len(dacWave3))
print(seg_wave_points_CH3)
#-----------------end CH3 AWG Waveform--------------------------------------
  
plt.plot(dacWave3)

In [ ]:
# Download it to segment 1 of channel 3
inst.send_scpi_cmd(":INST:CHAN 3") 
inst.send_scpi_cmd( ":TRAC:DEF 4," + str(len(dacWave3)), paranoia_level)
inst.send_scpi_cmd( ":TRAC:SEL 4", paranoia_level)

inst.write_binary_data(':TRAC:DATA', dacWave3)
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

## Define Seg 5 - CW - DC Remainder

In [ ]:

segLenDC3 = seg_wave_points_CH3-segLen3 #DC # Period_LEN-4800
#Set DC
dacWaveDC3 = np.zeros(segLenDC3)       
dacWaveDC3 = dacWaveDC3+(max_dac/2)
dacWaveDC3 = dacWaveDC3.astype(np.uint16)
   
inst.send_scpi_cmd(":INST:CHAN 3") 
# Set Data as Task 5
inst.send_scpi_cmd(':TRAC:DEF 5, {0}'.format(segLenDC3)) #*y.itemsize))
inst.send_scpi_cmd(':TRAC:SEL 5')
# download the waveform to the selected Task
inst.write_binary_data(':TRAC:DATA', dacWaveDC3)
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)


## Dowload Seg 1 (IDLE DC) for Chan 3

In [ ]:
# Build waveform (triangle wave)
segLenDCL3= 2048
y = np.ones(segLenDCL3)* half_dac
y = y.astype(data_type)
y.reshape(-1)


inst.send_scpi_cmd(":INST:CHAN 3") 
print(segLen)
# Set Data as Task 1
#inst.send_scpi_cmd(':TRAC:DEF 1, {0}'.format(segLen*y.itemsize))
inst.send_scpi_cmd(':TRAC:DEF 1, {0}'.format(segLenDCL3)) #*y.itemsize))
inst.send_scpi_cmd(':TRAC:SEL 1')
# download the waveform to the selected Task
inst.write_binary_data(':TRAC:DATA', y)
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)


## Define Marker Ch1 for Seg 2 (only Mark1)

In [ ]:
print('Build marker-data for sinus wave (segment 2) and download it to Proteus device.')

#seg_mark1_bytes = (896//8)+0 #len(dacWave) // 8
seg_mark1_bytes = len(dacWave)//8+0 #len(dacWave) // 8
marker1_len = (segLen//8)+1

mark1= np.ones(seg_mark1_bytes, np.uint8)
mark1[marker1_len:]=2

print(seg_mark1_bytes)
print(len(mark1))

## Define Marker Ch1 for Seg 3 (only Mar1 & Mark2)

In [ ]:
seg_mark2_bytes = segLenR//8 #len(dacWave) // 8 
mark2 = np.ones(seg_mark2_bytes, np.uint8)*2


print(segLenR)
print(seg_mark2_bytes)

In [ ]:
inst.send_scpi_cmd(":INST:CHAN 1")    
inst.send_scpi_cmd( ":TRAC:SEL 2", paranoia_level)    
inst.write_binary_data(':MARK:DATA',mark1)

inst.send_scpi_cmd(":INST:CHAN 1")
inst.send_scpi_cmd( ":TRAC:SEL 3", paranoia_level)    
inst.write_binary_data(':MARK:DATA',mark2)
   
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

## Configure Tasks Channel1

In [ ]:
tak_half_dac = 0x80

inst.send_scpi_cmd(':INST:CHAN 1')          # Select channel
inst.send_scpi_cmd(':TASK:COMP:LENG 3')     ### Set 5 Tasks in this Task Table

#TASK 1 - play a DC level and free run trigger (no trigger)
inst.send_scpi_cmd(':TASK:COMP:SEL 1')        #TASK1 composer
inst.send_scpi_cmd(':TASK:COMP:ENAB INT')    # CPU trigger is the task enable
inst.send_scpi_cmd(':TASK:COMP:SEGM 1')      # play segment 1 (DC)
inst.send_scpi_cmd(':TASK:COMP:IDLE:LEV {0}'.format(tak_half_dac))  # DC level while waiting to a trigger
inst.send_scpi_cmd(':TASK:COMP:NEXT1 2')     # the next task is 2, go to TASK2   

#TASK 2 -     
inst.send_scpi_cmd(':TASK:COMP:SEL 2')
inst.send_scpi_cmd(':TASK:COMP:ENAB NONE')    # NONE trigger is the task enable
inst.send_scpi_cmd(':TASK:COMP:SEGM 2' )       # play segment 2 (Signal)
inst.send_scpi_cmd(':TASK:COMP:TYPE START' )
inst.send_scpi_cmd(':TASK:COMP:SEQ 1') # sequence loop = AvgCount or other
#TODO - comment above
#inst.send_scpi_cmd(':TASK:COMP:SEQ 1')  #   loop set for 
inst.send_scpi_cmd(':TASK:COMP:DTRigger ON')   # issue a trigger to the ADC at start of segment
#inst.send_scpi_cmd(':TASK:COMP:IDLE:LEV {0}'.format(task_half_dac))  # DC level while waiting to a trigger
inst.send_scpi_cmd(':TASK:COMP:LOOP 1' )       # repeat the segment 1 time
inst.send_scpi_cmd(':TASK:COMP:NEXT1 3')       # the next task is 3


#TASK 3 - 
inst.send_scpi_cmd(':TASK:COMP:SEL 3')
inst.send_scpi_cmd(':TASK:COMP:SEGM 3' )       # play segment 3 (DC)
inst.send_scpi_cmd(':TASK:COMP:TYPE END' )
inst.send_scpi_cmd(':TASK:COMP:LOOP 1' )       # repeat the segment 1 time
#inst.send_scpi_cmd(':TASK:COMP:IDLE:LEV {0}'.format(task_half_dac))  # DC level while waiting to a trigger
#inst.send_scpi_cmd(':TASK:COMP:DTRigger ON')   # issue a trigger to the ADC at start of segment
#inst.send_scpi_cmd(':TASK:COMP:NEXT1 1')       # the next task is 1
#TODO - comment above
inst.send_scpi_cmd(':TASK:COMP:NEXT1 2')       # the next task is 2 for continous loop


inst.send_scpi_cmd(':TASK:COMP:WRIT' )          #write task sequence to memory
print('Downloading Task table of channel 1')




resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)


In [ ]:


resp = inst.send_scpi_query(':TASK:COMP:SEQ?')
print(resp)

# Configure Tasks Channel3

In [ ]:
tak_half_dac = 0x80

inst.send_scpi_cmd(':INST:CHAN 3')          # Select channel
inst.send_scpi_cmd(':TASK:COMP:LENG 3')     # Set 5 Tasks in this Task Table

#TASK 1 - play a DC level and free run trigger (no trigger)
inst.send_scpi_cmd(':TASK:COMP:SEL 1')        #TASK1 composer
inst.send_scpi_cmd(':TASK:COMP:ENAB INT')    # CPU trigger is the task enable
inst.send_scpi_cmd(':TASK:COMP:SEGM 1')      # play segment 1 (DC)
inst.send_scpi_cmd(':TASK:COMP:IDLE:LEV {0}'.format(tak_half_dac))  # DC level while waiting to a trigger
inst.send_scpi_cmd(':TASK:COMP:NEXT1 2')     # the next task is 2, go to TASK2   

#TASK 2 -     
inst.send_scpi_cmd(':TASK:COMP:SEL 2')
inst.send_scpi_cmd(':TASK:COMP:ENAB NONE')    # NONE trigger is the task enable
inst.send_scpi_cmd(':TASK:COMP:SEGM 4' )       # play segment 2 (Signal)
inst.send_scpi_cmd(':TASK:COMP:TYPE START' )
inst.send_scpi_cmd(':TASK:COMP:SEQ 1') # sequence loop = AvgCount or other
#TODO - comment above
#inst.send_scpi_cmd(':TASK:COMP:SEQ 1')  # For continous loop set 1
# No need to issue two adc triggers
#inst.send_scpi_cmd(':TASK:COMP:DTRigger OFF')   # issue a trigger to the ADC at start of segment
#inst.send_scpi_cmd(':TASK:COMP:IDLE:LEV {0}'.format(task_half_dac))  # DC level while waiting to a trigger
inst.send_scpi_cmd(':TASK:COMP:LOOP 1' )       # repeat the segment 1 time
inst.send_scpi_cmd(':TASK:COMP:NEXT1 3')       # the next task is 3


#TASK 3 - 
inst.send_scpi_cmd(':TASK:COMP:SEL 3')
inst.send_scpi_cmd(':TASK:COMP:SEGM 5' )       # play segment 3 (DC)
inst.send_scpi_cmd(':TASK:COMP:TYPE END' )
inst.send_scpi_cmd(':TASK:COMP:LOOP 1' )       # repeat the segment 1 time
#inst.send_scpi_cmd(':TASK:COMP:IDLE:LEV {0}'.format(task_half_dac))  # DC level while waiting to a trigger
#inst.send_scpi_cmd(':TASK:COMP:DTRigger ON')   # issue a trigger to the ADC at start of segment
#inst.send_scpi_cmd(':TASK:COMP:NEXT1 1')       # the next task is 1
#TODO - comment above
inst.send_scpi_cmd(':TASK:COMP:NEXT1 2')       # the next task is 2 for continous loop
    
   
    
inst.send_scpi_cmd(':TASK:COMP:WRIT' )          #write task sequence to memory
print('Downloading Task table of channel 3')


resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)


# Average Mode Parameters 

In [ ]:
#Digitizer Average Mode Settings 
##################################
#Average Count up 1-16M

AvgCount=1000
#12000#1000

##################################
#Frame Size 1-10240
# frame size should be smaller than the generation period in the 5.4 GS/s, 600 ns=3240 points

frame_size = 2840    # must be divisable by 40, 

number_of_frames =1 # Should be fixed one.

DTDEL='207.4e-9' #Digitizer Trigger delay 

GEN_VRANGE = 0.25 #P-P
ADC_RANGE = 'HIGH' #Adc range HIGH' #'MED', 'LOW'

# Setup Digitizer

In [ ]:

## New Average Mode registers:
inst.send_scpi_cmd(':DIG:ACQuire:AVERage:STAT ON')
inst.send_scpi_cmd(':DIG:ACQuire:AVERage:COUNt {0}'.format(AvgCount))

print("Set Digitizer: SINGLE mode; internal Trigger")
inst.send_scpi_cmd(':DIG:MODE SINGLE')
inst.send_scpi_cmd(':DIG:FREQ 5400MHZ')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)
inst.send_scpi_cmd(':DIG:DDC:CLKS AWG')

inst.send_scpi_cmd (':DIG:CHAN:RANG {}'.format(ADC_RANGE)) #set ADC range to 1000mV P2P
resp = inst.send_scpi_query(':DIG:CHAN:RANG?')
if resp == 'HIGH': 
     VRange = 0.5
elif resp == 'MED': 
    VRange = 0.4
elif resp == 'LOW': 
    VRange = 0.25
print(resp+" VRange: "+str(VRange))

inst.send_scpi_cmd(':DIG:ACQuire:FRAM:DEF {0}, {1}'.format(number_of_frames, frame_size))
inst.send_scpi_cmd(':DIG:ACQuire:FRAM:CAPT 1, 1')

# Set Trigger level to 0.2V
#inst.send_scpi_cmd(':DIG:TRIG:LEV1 -0.1')

# Enable capturing data from channel 1
inst.send_scpi_cmd(':DIG:CHAN:SEL 1')
inst.send_scpi_cmd(':DIG:CHAN:STATE ENAB')
# Select the internal-trigger as start-capturing trigger:
inst.send_scpi_cmd(':DIG:TRIG:SOURCE TASK1') #was TASK1



#Delay from Trig to ADC, now is 0.

##### <<< Need to select channel 1 delay>>>> ###
inst.send_scpi_cmd(':INST:CHAN 1') 
inst.send_scpi_cmd(':DIG:TRIG:AWG:TDEL {}'.format(DTDEL) )#150.0e-9'
resp = inst.send_scpi_query(':DIG:TRIG:AWG:TDEL?')
print("Trigger Delay: "+resp)


In [ ]:
resp = inst.send_scpi_query(':DIG:MODE?')
print("Digitizer Mode:"+resp)

resp = inst.send_scpi_query(':DIG:ACQuire:AVERage:STAT?')
print("Average Mode:"+resp)

resp = inst.send_scpi_query(':DIG:ACQuire:AVERage:COUNt?')
print("Average Count:"+resp)

resp = inst.send_scpi_query(':SHELL:REG? 0x004060AC') 
print("Avg Sum shift:"+resp)
AvgShiftMSB = resp

resp = inst.send_scpi_query(':DIG:ACQuire:FRAM:DEF?') 
print ("Frame size:"+ str(frame_size))
print(resp[2:])


# Start Channel 1 at Task mode

In [ ]:
# Enable channel 1
# Set Channel 1 to run task function

inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd('FUNC:MODE TASK')
inst.send_scpi_cmd(':SOUR:VOLT {}'.format(GEN_VRANGE)) #<<<< Output P-P Voltage.
 
inst.send_scpi_cmd(':MARK:SEL 1')
inst.send_scpi_cmd(':MARK:STAT ON')

inst.send_scpi_cmd(':MARK:SEL 2')
inst.send_scpi_cmd(':MARK:STAT ON')
inst.send_scpi_cmd(':OUTP ON')



In [ ]:
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':MARK:SEL 2')
resp = inst.send_scpi_query('FUNC:MODE?')
print(resp)
resp = inst.send_scpi_query(':MARK:STAT?')
print(resp)

# Start Channel 3 at Task mode

In [ ]:
# Enable channel 3
inst.send_scpi_cmd(':INST:CHAN 3') #stops the trigger!
inst.send_scpi_cmd('FUNC:MODE TASK')
inst.send_scpi_cmd(':SOUR:VOLT {}'.format(GEN_VRANGE))  #<<<< Output P-P Voltage.
inst.send_scpi_cmd(':OUTP ON')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

In [ ]:
resp = inst.send_scpi_query('FUNC:MODE?')
print(resp)

## Start Sampling Sequence

In [ ]:
# Stop the digitizer's capturing machine (to be on the safe side)
inst.send_scpi_cmd(':DIG:INIT OFF')


inst.send_scpi_cmd(':DIG:INIT ON')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)



## Trigger Gen Signal & Acquire Data

In [ ]:

## Currently Generates 1 M signal events.

inst.send_scpi_cmd(':TRIGger:COUPle 1')
inst.send_scpi_cmd('*TRG')


## Read Data

In [ ]:
#"<frame-done-bit>,<all-frames-done-bit>,<counter-busy-bit>,<frames-counter>"
frameRx = 0
print("Waiting for acquire to complete.")
times = 0
while (frameRx != 1):
    resp = inst.send_scpi_query(':DIG:ACQuire:FRAM:STATus?')
    framesParam = resp.split(",")
    frameRx = int(framesParam[3])
    times += 1
    timelib.sleep(0.1)
    if times > 100:
        break
        # end time and print time
print(resp)


if (times<100):
    print("Digitizer: Finish Acquisition")
else:
    print('Could not finish acquire data- try again')

In [ ]:
inst.send_scpi_cmd(':DIG:INIT OFF')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

inst.send_scpi_cmd(':DIG:DATA:SEL ALL')
inst.send_scpi_cmd(':DIG:DATA:TYPE FRAM')
resp = inst.send_scpi_query(':DIG:DATA:SIZE?')
#print(resp)
num_bytes = np.uint32(resp)
print('Total size in bytes: ' + resp)
print('Total samples: {}'.format(num_bytes//4))

# Read the data that was captured by channel 1:
inst.send_scpi_cmd(':DIG:CHAN:SEL 1')

wavlen = frame_size #num_bytes *2
if (wavlen != num_bytes//4):
  print ("Read size doesn't match frame size ")
#wav1 = np.zeros(wavlen, dtype=np.uint16)
wav1 = np.zeros(wavlen, dtype=np.uint32)

rc = inst.read_binary_data(':DIG:DATA:READ?', wav1, num_bytes)

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)
print("read data from DDR")


##  Plot Raw Data -  Binary Offset of 28 bit.

In [ ]:
digFreq=5400e6
ns100Samples = round(100e-9*digFreq)
maxAvg = np.max(wav1)
minAvg = np.min(wav1)


fig, axs = plt.subplots(1)
axs.plot(wav1)


xcoords = range(0,frame_size, ns100Samples)
for ix, xc in enumerate(xcoords):
    plt.vlines(x=xc, ymin=minAvg, ymax=maxAvg, colors='0.75', linestyles='dashed', label='{}ns'.format(100*ix))
    
plt.ylabel('ADC Count')
plt.xlabel('Frame')
plt.legend()

fig.suptitle('Count:')


print ("Count:{0}, MSB Shift:{1}".format(AvgCount, AvgShiftMSB))
print("Max:{0}, Min:{1}, Delta:{2}".format(maxAvg,minAvg,maxAvg-minAvg))

### Normalize to Signed number 

In [ ]:
def MsbCutoff(AvgCount):
    for i in range(31,14,-1):
      if ((1 << i) & AvgCount) != 0:
            break;
    log2FrameFactor = 1 << (i - 15);
    return log2FrameFactor;

In [ ]:
wavNorm = np.add(wav1, - np.power(2,27)) * MsbCutoff(AvgCount)/AvgCount
maxAvg = np.max(wavNorm)
minAvg = np.min(wavNorm)

fig, axs = plt.subplots(1)
axs.plot(wavNorm)
for ix, xc in enumerate(xcoords):
    plt.vlines(x=xc, ymin=minAvg, ymax=maxAvg, colors='0.75', linestyles='dashed', label='{}ns'.format(100*ix))
 
plt.ylabel('Frame')
fig.suptitle('Count:')


print ("Count:{0}, MSB Shift:{1}".format(AvgCount, AvgShiftMSB))
print("Max:{0}, Min:{1}, Delta:{2}".format(maxAvg,minAvg,maxAvg-minAvg))

In [ ]:
wavNormVolt = wavNorm*VRange/4096

maxAvg = np.max(wavNormVolt)
minAvg = np.min(wavNormVolt)

fig, axs = plt.subplots(1)
axs.plot(wavNormVolt)
for ix, xc in enumerate(xcoords):
    plt.vlines(x=xc, ymin=minAvg, ymax=maxAvg, colors='0.75', linestyles='dashed', label='{}ns'.format(100*ix))
 
plt.ylabel('Frame')
fig.suptitle('Count:')

print("VGen Vpp: "+ str(GEN_VRANGE))
print("DIG-VRange Vpp: "+ "+" + str(VRange/2)+ " to -" + str(VRange/2))
print("Signal Vpp: {:.2f} V".format(maxAvg-minAvg))

In [ ]:
plt.plot(wavNormVolt[50:300])

In [ ]:
# del=200ns, 175+(50)
# del=203.7ns, 175+(50)
# del=204ns, 175+(50)

# del=207.4ns, 140+(50)


In [ ]:
Debug=False

import tempfile
import webbrowser

if Debug == True :
    channb = 1
    cmd = ':INST:CHAN {0}; :SYST:INF:REG?'.format(channb)
    html_str = inst.send_scpi_query(cmd, max_resp_len=200000)
    print(html_str)
    with tempfile.NamedTemporaryFile('w', delete=False, suffix='.html') as f:
        url = 'file://' + f.name
        f.write(html_str)
    webbrowser.open(url)

In [ ]:
#inst.close_instrument()
#admin.close_inst_admin()


In [ ]:
#inst.send_scpi_cmd(':INST:CHAN 1')
#inst.send_scpi_cmd(':MARK:SEL 1')
#inst.send_scpi_cmd(':MARK:STAT ON')
#inst.send_scpi_cmd(':MARK:DEL:COAR 0')
#resp=inst.send_scpi_query(':MARK:DEL:FINE? MIN')
#print(resp)
#inst.send_scpi_cmd(':MARK:DEL:FINE -1e-9')